In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [3]:
train_data = pd.read_csv('/kaggle/input/feedback-prize-effectiveness/train.csv')
train_data

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate
...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective


In [4]:
train_data['discourse_effectiveness'] = train_data['discourse_effectiveness'].astype('category')
train_data['new_eff'] = train_data['discourse_effectiveness'].cat.codes
train_data['discourse_type'] = train_data['discourse_type'].astype('category')
train_data['new_type'] = train_data['discourse_type'].cat.codes                                                   
train_X = train_data.copy()
train_Y = train_X.pop('new_eff')
train_X2 = train_X.pop('new_type')
train_X.pop('discourse_effectiveness')
train_X.pop('discourse_type')
train_X.pop('essay_id')
train_X.pop('discourse_id')
train_Y

0        0
1        0
2        0
3        0
4        0
        ..
36760    0
36761    0
36762    0
36763    2
36764    2
Name: new_eff, Length: 36765, dtype: int8

In [5]:
train_X = np.array(train_X)
train_Y = np.array(train_Y)
train_X2 = np.array(train_X2)
train_Y = convert_to_one_hot(train_Y, 3)
train_X2 = convert_to_one_hot(train_X2, 7)
train_Y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [6]:
train_X = np.reshape(train_X, (train_X.shape[0],))
maxLen = len(max(train_X, key=len).split())
maxLen

836

In [7]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('/kaggle/input/glovedata/glove.6B.100d.txt')

In [8]:
import numpy as np
import tensorflow
np.random.seed(0)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation, Concatenate, Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform
np.random.seed(1)

In [9]:
def sentences_to_indices(X, word_to_index, max_len, train_X2):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m,)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros([m,max_len])
    X_final = np.zeros([m,maxLen+7])
    print(X_indices.shape)
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words = X[i].lower().split()
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words

        for w in sentence_words:
            # if w exists in the word_to_index dictionary
            if w in word_to_index:
                # Set the (i,j)th entry of X_indices to the index of the correct word.
                X_indices[i, j] = word_to_index[w]
                # Increment j to j + 1
                j =  j+1
#         print(X_indices[i].shape)
#         print(train_X2[i].shape)
        X_final[i] = np.concatenate((X_indices[i], train_X2[i]), axis=0)
#         print(X_final.shape)
            
    return X_final

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_size = len(word_to_index) + 1              # adding 1 to fit Keras embedding (requirement)
    any_word = list(word_to_vec_map.keys())[0]
    emb_dim = word_to_vec_map[any_word].shape[0]    # define dimensionality of your GloVe word vectors (= 50)
      
    # Initialize the embedding matrix as a numpy array of zeros.
    # See instructions above to choose the correct shape.
    emb_matrix = np.zeros((vocab_size,emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct input and output sizes
    # Make it non-trainable.
    embedding_layer = Embedding(input_dim = vocab_size, output_dim = emb_dim, trainable=False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. 
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer



def ArgEff_Net(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    # Define sentence_indices as the input of the graph.
    # It should be of shape input_shape and dtype 'int32' (as it contains indices, which are integers).
    sentence_indices = Input(shape=input_shape, dtype='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    X1, X2 = tensorflow.split(sentence_indices, [836, 7], 1)
    X2 = tensorflow.cast(X2, tensorflow.float32)
    
    # Propagate sentence_indices through your embedding layer
    embeddings = embedding_layer(X1)   
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # The returned output should be a batch of sequences.
    X = Bidirectional(LSTM(units=128, return_sequences=True))(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(rate=0.5)(X) 
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # The returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(units=128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
#     X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with 5 units
    X = Dense(units=64)(X)
    X = Concatenate(axis=1)([X,X2])
    X = Dense(units=7)(X)
    # Concatanate the discourse_type one-hot encodings with the 5 component output
    # 3 units for 3 types of ardument effectiveness
    X = Dense(units=3)(X)
    # Output the probability
    X = Activation('softmax')(X)
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs = sentence_indices, outputs = X)
    
    return model

model = ArgEff_Net((maxLen+7,), word_to_vec_map, word_to_index)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tensorflow.keras.metrics.CategoricalCrossentropy()])

2022-11-17 07:48:57.840493: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 07:48:57.939299: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 07:48:57.940105: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-17 07:48:57.941309: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 843)]        0                                            
__________________________________________________________________________________________________
tf.split (TFOpLambda)           [(None, 836), (None, 0           input_1[0][0]                    
__________________________________________________________________________________________________
embedding (Embedding)           (None, 836, 100)     40000100    tf.split[0][0]                   
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 836, 256)     234496      embedding[0][0]                  
______________________________________________________________________________________________

In [10]:
X_train_indices = sentences_to_indices(train_X, word_to_index, maxLen, train_X2)
X_train_indices

(36765, 836)


array([[1.63745e+05, 3.60915e+05, 7.10900e+04, ..., 1.00000e+00,
        0.00000e+00, 0.00000e+00],
       [2.69798e+05, 2.54258e+05, 1.85457e+05, ..., 0.00000e+00,
        1.00000e+00, 0.00000e+00],
       [1.85457e+05, 3.57970e+05, 3.57212e+05, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [4.70680e+04, 1.92973e+05, 3.37302e+05, ..., 0.00000e+00,
        1.00000e+00, 0.00000e+00],
       [3.37267e+05, 9.05480e+04, 3.74163e+05, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.88481e+05, 1.06807e+05, 6.12900e+04, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00]])

In [11]:
from sklearn.model_selection import train_test_split
X_trainf, X_testf, y_trainf, y_testf = train_test_split(X_train_indices, train_Y, test_size=0.2)

In [12]:
model.fit(X_trainf, y_trainf, validation_data=(X_testf, y_testf), epochs = 9, batch_size = 15, shuffle=False)

2022-11-17 07:49:04.085500: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/9


2022-11-17 07:49:08.832055: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1961/1961 [==============================] - 227s 113ms/step - loss: 0.9639 - categorical_crossentropy: 0.9639 - val_loss: 0.9616 - val_categorical_crossentropy: 0.9616
Epoch 2/9
1961/1961 [==============================] - 219s 112ms/step - loss: 0.9592 - categorical_crossentropy: 0.9592 - val_loss: 0.9616 - val_categorical_crossentropy: 0.9616
Epoch 3/9
1961/1961 [==============================] - 220s 112ms/step - loss: 0.9590 - categorical_crossentropy: 0.9590 - val_loss: 0.9615 - val_categorical_crossentropy: 0.9615
Epoch 4/9
1961/1961 [==============================] - 219s 112ms/step - loss: 0.9588 - categorical_crossentropy: 0.9588 - val_loss: 0.9615 - val_categorical_crossentropy: 0.9615
Epoch 5/9
1961/1961 [==============================] - 219s 112ms/step - loss: 0.9587 - categorical_crossentropy: 0.9587 - val_loss: 0.9615 - val_categorical_crossentropy: 0.9615
Epoch 6/9
1961/1961 [==============================] - 220s 112ms/step - loss: 0.9586 - categorical_crossentropy: 0

In [13]:
test_data = pd.read_csv('/kaggle/input/feedback-prize-effectiveness/test.csv')
test_data

,discourse_id,essay_id,discourse_text,discourse_type
0,a261b6e14276,D72CB1C11673,Making choices in life can be very difficult. ...,Lead
1,5a88900e7dc1,D72CB1C11673,Seeking multiple opinions can help a person ma...,Position
2,9790d835736b,D72CB1C11673,it can decrease stress levels,Claim
3,75ce6d68b67b,D72CB1C11673,a great chance to learn something new,Claim
4,93578d946723,D72CB1C11673,can be very helpful and beneficial.,Claim
5,2e214524dbe3,D72CB1C11673,When making a decision there is a chance that ...,Evidence
6,84812fc2ab9f,D72CB1C11673,Everyone is different and may have more experi...,Evidence
7,c668ff840720,D72CB1C11673,Seeking others opinion can be very helpful and...,Claim
8,739a6d00f44a,D72CB1C11673,Taking other peoples advice and doing what the...,Evidence
9,bcfae2c9a244,D72CB1C11673,You can learn from others experiences by seeki...,Concluding Statement


In [14]:
test_data['discourse_type'] = test_data['discourse_type'].astype('category')
test_data['new_type'] = test_data['discourse_type'].cat.codes                                                   
test_X = test_data.copy()
test_X2 = test_X.pop('new_type')
test_X.pop('discourse_type')
test_X.pop('essay_id')
test_X.pop('discourse_id')
test_X

,discourse_text
0,Making choices in life can be very difficult. ...
1,Seeking multiple opinions can help a person ma...
2,it can decrease stress levels
3,a great chance to learn something new
4,can be very helpful and beneficial.
5,When making a decision there is a chance that ...
6,Everyone is different and may have more experi...
7,Seeking others opinion can be very helpful and...
8,Taking other peoples advice and doing what the...
9,You can learn from others experiences by seeki...


In [15]:
test_X = np.array(test_X)
test_X2 = np.array(test_X2)
test_X2 = convert_to_one_hot(test_X2, 7)
test_X

array([["Making choices in life can be very difficult. People often ask for advice when they can not decide on one thing. It's always good to ask others for their advice when making a choice. When you have multiple opinions you have the ability to make the best choice for yourself. "],
       ['Seeking multiple opinions can help a person make a better choice '],
       ['it can decrease stress levels '],
       ['a great chance to learn something new '],
       ['can be very helpful and beneficial. '],
       ['When making a decision there is a chance that you can become very stressed and overwhelmed. Being stressed can cause a person to give up on what they are doing. Having multiple opinions from people can make choosing an option less difficult and more easier. The information that you receive from others may have a special meaning in the future. Other peoples opinion can make a person feel confident in making the right choice. '],
       ['Everyone is different and may have more ex

In [16]:
test_X = np.reshape(test_X, (test_X.shape[0],))
X_test_indices = sentences_to_indices(test_X, word_to_index, maxLen, train_X2)
X_test_indices

(10, 836)


array([[2.31552e+05, 9.97750e+04, 1.88481e+05, ..., 1.00000e+00,
        0.00000e+00, 0.00000e+00],
       [3.23399e+05, 2.52664e+05, 2.70694e+05, ..., 0.00000e+00,
        1.00000e+00, 0.00000e+00],
       [1.93716e+05, 9.05480e+04, 1.18964e+05, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [3.23399e+05, 2.72587e+05, 2.70692e+05, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [3.52025e+05, 2.72583e+05, 2.80955e+05, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [3.94475e+05, 9.05480e+04, 2.19109e+05, ..., 1.00000e+00,
        0.00000e+00, 0.00000e+00]])

In [17]:
Y_pred = model.predict(X_test_indices)
Y_pred

array([[0.5576536 , 0.28586677, 0.15647966],
       [0.7000424 , 0.18754137, 0.11241616],
       [0.5847583 , 0.28042296, 0.13481878],
       [0.50573   , 0.22582953, 0.2684405 ],
       [0.64339626, 0.2351025 , 0.12150124],
       [0.53900373, 0.28010362, 0.18089266],
       [0.50573   , 0.22582953, 0.2684405 ],
       [0.64339626, 0.2351025 , 0.12150124],
       [0.57910305, 0.24059653, 0.18030043],
       [0.5576536 , 0.28586677, 0.15647966]], dtype=float32)

In [18]:
submission = test_data[['discourse_id']].copy()
submission

,discourse_id
0,a261b6e14276
1,5a88900e7dc1
2,9790d835736b
3,75ce6d68b67b
4,93578d946723
5,2e214524dbe3
6,84812fc2ab9f
7,c668ff840720
8,739a6d00f44a
9,bcfae2c9a244


In [19]:
submission['Ineffective'] = Y_pred[:, 2]
submission['Adequate'] = Y_pred[:, 0]
submission['Effective'] = Y_pred[:, 1]
submission

,discourse_id,Ineffective,Adequate,Effective
0,a261b6e14276,0.156480,0.557654,0.285867
1,5a88900e7dc1,0.112416,0.700042,0.187541
2,9790d835736b,0.134819,0.584758,0.280423
3,75ce6d68b67b,0.268441,0.505730,0.225830
4,93578d946723,0.121501,0.643396,0.235103
5,2e214524dbe3,0.180893,0.539004,0.280104
6,84812fc2ab9f,0.268441,0.505730,0.225830
7,c668ff840720,0.121501,0.643396,0.235103
8,739a6d00f44a,0.180300,0.579103,0.240597
9,bcfae2c9a244,0.156480,0.557654,0.285867


In [20]:
submission.to_csv('submission.csv', index=False)